Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Perform a tuning task with schedulers
## 0. Introduction
In this notebook, we show how to perform tuning tasks in `flaml.tune` with two different types of schedulers.
- Scheduler type 1, the authentic scheduler included in FLAML. It assumes the performance of a particular configuration becomes better with the increase of resources.
- Scheduler type 2: the schedulers which assume the knowledge of the maximum resource to use and assume the best performance can be achieved at the max resource.
Most existing schedulers are of this type, including ASHA, Hyperband, BOHB. 



In [ ]:
# requirements to run the examples in this notebook
!pip install flaml[ray];


## 1. Configure the objective functions according to the types of schedulers
We start by defining a vanilla objective as `simple_obj`, and then introduce how this objective function should be revised according to the type of the scheduler.

Background about the tuning task:
The tuning task is an optimization problem in which we want to maximize a half 2-sphere's projection on a vector in a particular 3-dimensional cube $S$, say the names of the three coordinates are "x", "y", "z".

Suppose you do not know how to solve the optimization problem directly, but know how to sample data points from the half 2-sphere uniformly. Then you can get an approximation of the projection value on any vector in the search space. A good approximation of the ground-truth optimization objective makes it possible for you to solve the original optimization problem. The evaluation in `simple_obj` performs such an approximation, which has the following two properties:
1. The quality of the approximation is resource-dependent, and the more resource is used, the better the approximation.
2. The computational cost of the approximation increases monotonically the amount of resource used.

In [91]:
import numpy as np
from flaml import tune
import time

def rand_vector_unit_sphere(dim):
    """this function allows you to generate
    points that uniformly distribute on 
    the (dim-1)-sphere.
    """
    vec = np.random.normal(0, 1, dim)
    mag = np.linalg.norm(vec)
    # time.sleep(0.1)
    return vec / mag

def simple_obj(config, resource=500000):
    config_value_vector = np.array([config["x"], config["y"], config["z"]])
    score_sequence = []
    for i in range(resource):
        a = rand_vector_unit_sphere(3)
        a[2] = abs(a[2])
        point_projection = np.dot(config_value_vector, a)
        score_sequence.append(point_projection)
    score_avg = np.mean(np.array(score_sequence))
    score_std = np.std(np.array(score_sequence))
    score_lb = score_avg - 1.96 * score_std/np.sqrt(resource)
    tune.report(samplesize = resource, sphere_projection = score_lb)

In [92]:
"""Define an evaluation function which can report intermediate result"""
def obj_w_intermediate_report(resource, config):
    config_value_vector = np.array([config["x"], config["y"], config["z"]])
    score_sequence = []
    for i in range(resource):
        a = rand_vector_unit_sphere(3)
        a[2] = abs(a[2])
        point_projection = np.dot(config_value_vector, a)
        score_sequence.append(point_projection)
        if (i + 1) % 5000 == 0:
            score_avg = np.mean(np.array(score_sequence))
            score_std = np.std(np.array(score_sequence))
            score_lb = score_avg - 1.96 * score_std/np.sqrt(i+1)
            tune.report(samplesize = i + 1, sphere_projection=score_lb)

"""Define evaluation functions which take resource according to the 
   suggested value in config"""
def obj_w_suggested_resource(resource_attr, config):
    resource = config[resource_attr]
    simple_obj(config, resource)

def obj_w_suggested_resource_w_intermediate_report(resource_attr, config):
    resource = config[resource_attr]
    obj_w_intermediate_report(resource, config)

## 2. Specify the tuning task

In [ ]:
#  define the search space
search_space = {
            "x": tune.uniform(5, 20),
            "y": tune.uniform(0, 10),
            "z": tune.uniform(0, 10),
        }

In [93]:
# specify the tuning task with different types of schedulers
def test_scheduler(scheduler=None, use_flaml_scheduler=False):
    from functools import partial
    resource_attr = "samplesize"
    max_resource = 500000

    # specify the objective functions
    if not use_flaml_scheduler and scheduler is None:
        evaluation_obj = simple_obj
    elif use_flaml_scheduler and scheduler is None:
        evaluation_obj = partial(obj_w_suggested_resource, resource_attr)
    elif not use_flaml_scheduler and scheduler:
        evaluation_obj = partial(obj_w_intermediate_report, max_resource)
    else:
        evaluation_obj = partial(obj_w_suggested_resource_w_intermediate_report, resource_attr)

    analysis = tune.run(
        evaluation_obj,
        config=search_space,
        metric="sphere_projection",
        mode="max",
        verbose=1,
        resource_attr=resource_attr,
        use_flaml_scheduler=use_flaml_scheduler,
        scheduler=scheduler,
        max_resource=max_resource,
        min_resource=100,
        reduction_factor=2,
        time_budget_s=10,
        num_samples=300,
    )

    print("Best hyperparameters found were: ", analysis.best_config)
    # print(analysis.get_best_trial)
    return analysis.best_config

## 3. Result evaluation
According to calculus, the ground-truth optimization objective is `config["z"]/2`, which means that all the configurations that have the largest value on the 3rd, i.e., the "z" coordinate are the optimal solutions. According to the search space, the largest value config[`z`] can take is 10, so 5 is the optimal value. We now compare the absolute difference between 5 and the expected optimal value `config["z"]/2` according to the best configurations found in experiments with different schedulers.

In [94]:
best_config = test_scheduler()
print('No scheduler, test error:', abs(10/2 -best_config['z']/2) )

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2021-12-02 03:42:02,185] A new study created in memory with name: optuna
[flaml.tune.tune: 12-02 03:42:02] {416} INFO - trial 1 config: {'x': 8.9701674273961, 'y': 0.6316750485854095, 'z': 2.0444107098503093}
[flaml.tune.tune: 12-02 03:42:08] {416} INFO - trial 2 config: {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}


Best hyperparameters found were:  {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}
No scheduler, test error: 1.831758825368623


In [95]:
# use auto ASHA scheduler
best_config = test_scheduler('auto')
print('Auto ASHA scheduler, test error:', abs(10/2 -best_config['z']/2) )

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2021-12-02 03:42:15,467] A new study created in memory with name: optuna
[flaml.tune.tune: 12-02 03:42:15] {416} INFO - trial 1 config: {'x': 5.095596707171773, 'y': 6.1178606957572255, 'z': 0.655562465645696}
[flaml.tune.tune: 12-02 03:42:24] {416} INFO - trial 2 config: {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}


Best hyperparameters found were:  {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}
Auto ASHA scheduler, test error: 1.831758825368623


In [96]:
# use a custom ASHAScheduler with a smaller max resource
from ray.tune.schedulers import ASHAScheduler
my_scheduler = ASHAScheduler(time_attr="samplesize", max_t=1000, grace_period=50,reduction_factor=2)
best_config = test_scheduler(my_scheduler)
print('Custom ASHA scheduler, test error:', abs(10/2 -best_config['z']/2) )

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2021-12-02 03:42:33,229] A new study created in memory with name: optuna
[flaml.tune.tune: 12-02 03:42:33] {416} INFO - trial 1 config: {'x': 12.593349508841978, 'y': 0.14797564076601044, 'z': 0.7469513534682937}
[flaml.tune.tune: 12-02 03:42:42] {416} INFO - trial 2 config: {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}


Best hyperparameters found were:  {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754}
Custom ASHA scheduler, test error: 1.831758825368623


In [97]:
# use FLAML scheduler
best_config = test_scheduler(scheduler=None, use_flaml_scheduler=True)
print('FLAML scheduler, test error', abs(10/2 -best_config['z']/2) )

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2021-12-02 03:42:51,334] A new study created in memory with name: optuna
[flaml.tune.tune: 12-02 03:42:51] {416} INFO - trial 1 config: {'x': 11.620994821419593, 'y': 3.309160421378788, 'z': 9.329888805740406, 'samplesize': 100}
[flaml.tune.tune: 12-02 03:42:51] {416} INFO - trial 2 config: {'x': 16.56980964900119, 'y': 0.207519493594015, 'z': 6.336482349262754, 'samplesize': 100}
[flaml.tune.tune: 12-02 03:42:51] {416} INFO - trial 3 config: {'x': 16.232058238079176, 'y': 4.9850701230259045, 'z': 2.2479664553084766, 'samplesize': 100}
[flaml.tune.tune: 12-02 03:42:51] {416} INFO - trial 4 config: {'x': 7.97094297139436, 'y': 7.605307121989587, 'z': 1.691108365625354

Best hyperparameters found were:  {'x': 7.346432298553975, 'y': 9.178329973271207, 'z': 10, 'samplesize': 200}
FLAML scheduler, test error 0.0
